# Knight's Tour example

Example problems with a Knight's Tour on the chess board.

In [ ]:
%pip install matplotlib numpy scipy

In [ ]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import scipy
import sys

sys.path.append(os.path.abspath(".."))

## Number of options for tour between oppsite corners

In 8 steps, how many paths can a knight take between opposite corners?

In [ ]:
import knight_tour
import simprob


def advance_knight_counts(counts):
    return scipy.signal.convolve(counts, knight_tour.knight_moves_kernel, mode="same")


start = np.zeros(knight_tour.BOARD_SHAPE, dtype=int)
start[0, 0] = 1
n_steps = 8
for state in simprob.simulate(
    start, [simprob.Iteration(transition=advance_knight_counts)] * n_steps
):
    plt.figure(figsize=(2, 2))
    plt.imshow(state)
    plt.colorbar()
    plt.show()
print(f"Number of paths of {n_steps} steps between opposite corners: {state[-1, -1]}")

## Infer path from partial observations.

Given sporadic observations of Knight row, column, diagonal or quadrant at specific times,
could we infer where the knight have been?

### Randomize path

In [ ]:
path = list(itertools.islice(knight_tour.random_knight_path(), 15))
plt.axis("equal")
plt.title("The knight's tour")
_ = plt.plot(*zip(*path), ".-")

### Randomize observations

In [ ]:
observations = []
for x, y in path:
    kind = random.choice(["row", "col", "diag0", "diag1", "quadrant"])
    obs = np.zeros(knight_tour.BOARD_SHAPE, dtype=bool)
    if kind == "row":
        obs[:, y] = True
    elif kind == "col":
        obs[x] = True
    elif kind == "diag0":
        for i in range(-min(x, y), 8 - max(x, y)):
            obs[x + i, y + i] = True
    elif kind == "diag1":
        for i in range(-min(7 - x, y), 8 - max(7 - x, y)):
            obs[x - i, y + i] = True
    else:
        obs[
            slice(4) if x < 4 else slice(4, None), slice(4) if y < 4 else slice(4, None)
        ] = True
    observations.append(obs)
observations = np.asarray(observations)

print("Partial observations of knight's position over time")
_, ax = plt.subplots(ncols=len(observations))
for a, o, (x, y) in zip(ax, observations, path):
    o = o.astype(int)
    o[x, y] += 1
    a.imshow(o)
    a.set_xticks([])
    a.set_yticks([])

### Infer path using simprob

In [ ]:
import simprob.back_sim as bwd


class KnightTransition:
    def __call__(self, mask: np.ndarray) -> np.ndarray:
        return (
            scipy.signal.convolve(
                mask.astype(int), knight_tour.knight_moves_kernel, mode="same"
            )
            > 0
        )

    @property
    def inv(self):
        return self


inferred = np.asarray(
    list(
        bwd.simulate_fwd_bwd(
            observations[0],
            [
                simprob.Iteration(transition=KnightTransition(), observation=o)
                for o in observations[1:]
            ],
            np.ones(knight_tour.BOARD_SHAPE, dtype=bool),
        )
    )
)

print("Inference of knight's possible positions from observations")
_, ax = plt.subplots(ncols=len(inferred))
for a, i in zip(ax, inferred + observations.astype(int)):
    a.imshow(i)
    a.set_xticks([])
    a.set_yticks([])

## Infering the path with probabilities

See `Hidden Markov Model.ipynb` for an example